# Planification du raccordement électrique de bâtiments

In [11]:
#pip install pandas openpyxl geopandas matplotlib

In [3]:
import pandas as pd

#les deux fichiers CSV
batiments = pd.read_csv("batiments.csv")
infra = pd.read_csv("infra.csv")

#valeurs distinctes
print("Valeurs distinctes de type_batiment :")
print(batiments["type_batiment"].dropna().unique())

print("\nValeurs distinctes de type_infra :")
print(infra["type_infra"].dropna().unique())


Valeurs distinctes de type_batiment :
['habitation' 'hôpital' 'école']

Valeurs distinctes de type_infra :
['aerien' 'fourreau' 'semi-aerien']


In [4]:
# shapefiles 
try:
    gdf_infrastructures_cols = gpd.read_file("infrastructures.shp")
    print("Columns in infrastructures.shp:")
    print(gdf_infrastructures_cols.columns)

    gdf_batiments_cols = gpd.read_file("batiments.shp")
    print("\nColumns in batiments.shp:")
    print(gdf_batiments_cols.columns)
except Exception as e:
    print(f"Error loading shapefiles: {e}")

Error loading shapefiles: name 'gpd' is not defined


In [5]:
import os
import unicodedata
import pandas as pd
import geopandas as gpd
from dataclasses import dataclass, field
from typing import List, Dict, Set, Optional

In [6]:

# paramètres 

COST_PER_M = {
    "aerien": 500.0,
    "semi-aerien": 750.0,
    "semi-aérien": 750.0, 
    "fourreau": 900.0,
}
HOURS_PER_M = {
    "aerien": 2.0,
    "semi-aerien": 4.0,
    "semi-aérien": 4.0,
    "fourreau": 5.0,
}
WORKER_HOURLY = 300.0 / 8.0  
MAX_WORKERS_PER_INFRA = 4
HOSPITAL_AUTONOMY_H = 20.0
HOSPITAL_MARGIN = 0.20
HOSPITAL_DEADLINE_H = HOSPITAL_AUTONOMY_H * (1.0 - HOSPITAL_MARGIN)  # 16h

PHASE_BUDGET_SPLITS = [0.40, 0.20, 0.20, 0.20]
BUDGET_TOL = 0.03  # 3%

def _strip_accents_lower(s: Optional[str]) -> str:
    if s is None:
        return ""
    return "".join(c for c in unicodedata.normalize("NFD", s)
                   if unicodedata.category(c) != "Mn").lower()


#modèle
@dataclass
class Infrastructure:
    id: str
    etat: str  # infra_intacte ou a_remplacer
    longueur: float
    type_infra: str = ""  # aerien ou semi-aerien ou fourreau
    batiments_desservis: Set[str] = field(default_factory=set)
    nb_maisons_total: int = 0

    cout_mat: float = 0.0
    heures: float = 0.0
    cout_mo: float = 0.0
    cout_total: float = 0.0
    duree_eff: float = 0.0
    diff: float = 0.0
    est_reparee: bool = False
    phase_reparation: int = -1

    def precalc(self):
        t = self.type_infra
        cpm = COST_PER_M.get(t, None)
        hpm = HOURS_PER_M.get(t, None)
        if cpm is None or hpm is None:
            raise ValueError(f"type_infra inconnu pour l'infra {self.id}: '{t}'")

        self.cout_mat = self.longueur * cpm
        self.heures = self.longueur * hpm           
        self.cout_mo = self.heures * WORKER_HOURLY
        self.cout_total = self.cout_mat + self.cout_mo
        self.duree_eff = self.heures / MAX_WORKERS_PER_INFRA if MAX_WORKERS_PER_INFRA > 0 else float('inf')

        denom = self.nb_maisons_total if self.nb_maisons_total > 0 else None
        self.diff = (self.cout_total * self.heures) / denom if denom else float('inf')
        return self

    def est_operationnelle(self) -> bool:
        return self.etat == "infra_intacte" or self.est_reparee

@dataclass
class Batiment:
    id: str
    nb_maisons: int
    type_batiment: str = ""  # habitation ou hôpital ou école
    infrastructures: List[Infrastructure] = field(default_factory=list)
    difficulte_totale: float = 0.0
    phase: int = -1

    def calculer_difficulte(self):
        non_op = [i for i in self.infrastructures if not i.est_operationnelle()]
        self.difficulte_totale = sum(i.diff for i in non_op)
        return self.difficulte_totale

    def est_raccorde(self) -> bool:
        return all(i.est_operationnelle() for i in self.infrastructures)


# planificateur
class PlanificateurRaccordement:
    def __init__(self, xlsx_path, shp_infra_path, shp_bat_path,
                 batiments_csv="batiments.csv", infra_csv="infra.csv"):
        self.xlsx_path = xlsx_path
        self.shp_infra_path = shp_infra_path
        self.shp_bat_path = shp_bat_path
        self.batiments_csv = batiments_csv
        self.infra_csv = infra_csv

        self.df_reseau = None
        self.gdf_infrastructures = None
        self.gdf_batiments = None
        self.df_bats = None
        self.df_infras = None

        self.infrastructures: Dict[str, Infrastructure] = {}
        self.batiments: Dict[str, Batiment] = {}
        self.plan_raccordement: List[Dict] = []

    # chargement data
    def charger_donnees(self):
        print("Chargement des données...")
        self.df_reseau = pd.read_excel(self.xlsx_path)
        self.df_bats = pd.read_csv(self.batiments_csv)
        self.df_infras = pd.read_csv(self.infra_csv)

        try:
            self.gdf_infrastructures = gpd.read_file(self.shp_infra_path)
            self.gdf_batiments = gpd.read_file(self.shp_bat_path)
        except Exception:
            self.gdf_infrastructures = None
            self.gdf_batiments = None

        print(f"{len(self.df_reseau)} lignes de réseau chargées")

        #  infra_id, id_infra, id_batiment, batiment_id
        infra_id_col = "infra_id" if "infra_id" in self.df_infras.columns else ("id_infra" if "id_infra" in self.df_infras.columns else None)
        bat_id_col = "id_batiment" if "id_batiment" in self.df_bats.columns else ("batiment_id" if "batiment_id" in self.df_bats.columns else None)
        if infra_id_col is None or bat_id_col is None:
            raise ValueError("Colonnes d'identifiants introuvables (infra_id/id_infra et id_batiment/batiment_id).")

        infra_type_map = dict(zip(self.df_infras[infra_id_col].astype(str), self.df_infras["type_infra"].astype(str)))
        bat_type_map = dict(zip(self.df_bats[bat_id_col].astype(str), self.df_bats["type_batiment"].astype(str)))

        # infras
        for infra_id in self.df_reseau['infra_id'].astype(str).unique():
            ligne = self.df_reseau[self.df_reseau['infra_id'].astype(str) == infra_id].iloc[0]
            type_infra = infra_type_map.get(infra_id, str(ligne.get('type_infra', '')))
            type_infra = _strip_accents_lower(type_infra)

            self.infrastructures[infra_id] = Infrastructure(
                id=infra_id,
                etat=str(ligne['infra_type']),
                longueur=float(ligne['longueur']),
                type_infra=type_infra
            )

        #batiments
        for bat_id in self.df_reseau['id_batiment'].astype(str).unique():
            data = self.df_reseau[self.df_reseau['id_batiment'].astype(str) == bat_id]
            nb = int(data['nb_maisons'].iloc[0])
            type_bat = bat_type_map.get(bat_id, "")
            bat = Batiment(id=bat_id, nb_maisons=nb, type_batiment=type_bat)

            for _, row in data.iterrows():
                infra = self.infrastructures[str(row['infra_id'])]
                bat.infrastructures.append(infra)
                infra.batiments_desservis.add(bat_id)
                infra.nb_maisons_total += nb

            self.batiments[bat_id] = bat

        # pré-calculs
        for infra in self.infrastructures.values():
            infra.precalc()
        for bat in self.batiments.values():
            bat.calculer_difficulte()

        print(f"{len(self.batiments)} bâtiments chargés")
        print(f"{len(self.infrastructures)} infrastructures chargées\n")

    # outil
    def _cout_restants(self) -> float:
        return sum(i.cout_total for i in self.infrastructures.values() if not i.est_operationnelle())

    def _infras_non_op_bat(self, bat: Batiment) -> List[Infrastructure]:
        non_op = [i for i in bat.infrastructures if not i.est_operationnelle()]
        return list({i.id: i for i in non_op}.values())

    #planification
    def planifier_raccordement(self):
        print("Début de la planification...\n")
        cout_total_global = 0.0

        #phase 0 : hopital
        hospital_candidates = [b for b in self.batiments.values()
                               if _strip_accents_lower(b.type_batiment) in ("hopital", "hopital ", "hôpital")]
        if hospital_candidates:
            hopital = hospital_candidates[0]
            infras_h = self._infras_non_op_bat(hopital)
            if infras_h:
                duree_phase0 = max(i.duree_eff for i in infras_h)
                ok_hopital_temps = duree_phase0 <= HOSPITAL_DEADLINE_H

                cout_phase_mat = sum(i.cout_mat for i in infras_h)
                cout_phase_mo = sum(i.cout_mo for i in infras_h)
                cout_phase_total = cout_phase_mat + cout_phase_mo
                heures_phase = sum(i.heures for i in infras_h)
                for i in infras_h:
                    i.est_reparee = True
                    i.phase_reparation = 0

                hopital.phase = 0
                cout_total_global += cout_phase_total

                self.plan_raccordement.append({
                    "Phase": 0,
                    "Bâtiment": hopital.id,
                    "Type bâtiment": hopital.type_batiment,
                    "Maisons": hopital.nb_maisons,
                    "Difficulté": round(hopital.difficulte_totale, 2),
                    "Infras réparées": ", ".join(i.id for i in infras_h),
                    "Coût matériel (phase)": round(cout_phase_mat, 2),
                    "Coût MO (phase)": round(cout_phase_mo, 2),
                    "Coût total (phase)": round(cout_phase_total, 2),
                    "Coût cumulé": round(cout_total_global, 2),
                    "Heures (phase)": round(heures_phase, 2),
                    "Durée critique (h)": round(duree_phase0, 2),
                    "Hopital_OK_≤16h": "OK" if ok_hopital_temps else "ALERTE"
                })
                for b in self.batiments.values():
                    b.calculer_difficulte()
            else:
                hopital.phase = 0
                self.plan_raccordement.append({
                    "Phase": 0,
                    "Bâtiment": hopital.id,
                    "Type bâtiment": hopital.type_batiment,
                    "Maisons": hopital.nb_maisons,
                    "Difficulté": 0.0,
                    "Infras réparées": "",
                    "Coût matériel (phase)": 0.0,
                    "Coût MO (phase)": 0.0,
                    "Coût total (phase)": 0.0,
                    "Coût cumulé": round(cout_total_global, 2),
                    "Heures (phase)": 0.0,
                    "Durée critique (h)": 0.0,
                    "Hopital_OK_≤16h": "OK"
                })

        # marquer déjà op
        for bat in self.batiments.values():
            if bat.phase == -1 and bat.est_raccorde():
                bat.phase = 0

        remaining = self._cout_restants()
        phase_caps = [split * remaining for split in PHASE_BUDGET_SPLITS]

        current_phase_num = 1
        batiments_restants = [b for b in self.batiments.values() if b.phase == -1]

        while current_phase_num <= 4 and batiments_restants:
            cap = phase_caps[current_phase_num - 1] if current_phase_num - 1 < len(phase_caps) else float('inf')
            tol = cap * BUDGET_TOL
            cout_phase_total = 0.0
            duree_phase_critique = 0.0

            placed_any = True
            while placed_any:
                for b in batiments_restants:
                    b.calculer_difficulte()

                def key_b(b: Batiment):
                    infras_needed = self._infras_non_op_bat(b)
                    cout_needed = sum(i.cout_total for i in infras_needed)
                    priority = 0
                    tb = _strip_accents_lower(b.type_batiment)
                    if tb in ("ecole", "école"):
                        priority = -1
                    return (b.difficulte_totale, cout_needed, priority, -b.nb_maisons)

                batiments_restants.sort(key=key_b)
                placed_any = False

                for b in list(batiments_restants):
                    infras_needed = self._infras_non_op_bat(b)
                    if not infras_needed:
                        b.phase = current_phase_num
                        batiments_restants.remove(b)
                        placed_any = True
                        self.plan_raccordement.append({
                            "Phase": current_phase_num,
                            "Bâtiment": b.id,
                            "Type bâtiment": b.type_batiment,
                            "Maisons": b.nb_maisons,
                            "Difficulté": round(b.difficulte_totale, 2),
                            "Infras réparées": "",
                            "Coût matériel (phase)": 0.0,
                            "Coût MO (phase)": 0.0,
                            "Coût total (phase)": 0.0,
                            "Coût cumulé": round(cout_total_global, 2),
                            "Heures (phase)": 0.0,
                            "Durée critique (h)": round(duree_phase_critique, 2),
                        })
                        continue

                    cout_mat = sum(i.cout_mat for i in infras_needed)
                    cout_mo = sum(i.cout_mo for i in infras_needed)
                    cout_needed = cout_mat + cout_mo
                    if cout_phase_total + cout_needed <= cap + tol or (cap == 0 and cout_phase_total == 0):
                        for i in infras_needed:
                            i.est_reparee = True
                            i.phase_reparation = current_phase_num

                        cout_phase_total += cout_needed
                        duree_crit_add = max((i.duree_eff for i in infras_needed), default=0.0)
                        duree_phase_critique = max(duree_phase_critique, duree_crit_add)
                        cout_total_global += cout_needed

                        b.phase = current_phase_num
                        batiments_restants.remove(b)
                        placed_any = True

                        self.plan_raccordement.append({
                            "Phase": current_phase_num,
                            "Bâtiment": b.id,
                            "Type bâtiment": b.type_batiment,
                            "Maisons": b.nb_maisons,
                            "Difficulté": round(b.difficulte_totale, 2),
                            "Infras réparées": ", ".join(i.id for i in infras_needed),
                            "Coût matériel (phase)": round(cout_mat, 2),
                            "Coût MO (phase)": round(cout_mo, 2),
                            "Coût total (phase)": round(cout_needed, 2),
                            "Coût cumulé": round(cout_total_global, 2),
                            "Heures (phase)": round(sum(i.heures for i in infras_needed), 2),
                            "Durée critique (h)": round(duree_phase_critique, 2),
                        })
                        break

                if not placed_any:
                    break

            current_phase_num += 1

        #sécu : reste éventuel en phase 5
        for b in batiments_restants:
            last_phase = 5
            infras_needed = self._infras_non_op_bat(b)
            cout_mat = sum(i.cout_mat for i in infras_needed)
            cout_mo = sum(i.cout_mo for i in infras_needed)
            cout_needed = cout_mat + cout_mo
            for i in infras_needed:
                i.est_reparee = True
                i.phase_reparation = last_phase
            cout_total_global += cout_needed
            b.phase = last_phase
            self.plan_raccordement.append({
                "Phase": last_phase,
                "Bâtiment": b.id,
                "Type bâtiment": b.type_batiment,
                "Maisons": b.nb_maisons,
                "Difficulté": round(b.difficulte_totale, 2),
                "Infras réparées": ", ".join(i.id for i in infras_needed),
                "Coût matériel (phase)": round(cout_mat, 2),
                "Coût MO (phase)": round(cout_mo, 2),
                "Coût total (phase)": round(cout_needed, 2),
                "Coût cumulé": round(cout_total_global, 2),
                "Heures (phase)": round(sum(i.heures for i in infras_needed), 2),
                "Durée critique (h)": round(max((i.duree_eff for i in infras_needed), default=0.0), 2),
            })

        print("Planification terminée.")
        print(f"Coût total estimé : {round(cout_total_global, 2)} €\n")

    # tableau
    def afficher_plan(self) -> pd.DataFrame:
        df = pd.DataFrame(self.plan_raccordement)
        preferred = ["Phase", "Bâtiment", "Type bâtiment", "Maisons", "Difficulté",
                     "Infras réparées", "Coût matériel (phase)", "Coût MO (phase)",
                     "Coût total (phase)", "Coût cumulé", "Heures (phase)", "Durée critique (h)", "Hopital_OK_≤16h"]
        cols = [c for c in preferred if c in df.columns] + [c for c in df.columns if c not in preferred]
        df = df[cols]
        print("Tableau du plan de raccordement :\n")
        print(df.to_string(index=False))
        return df

    #cartes par phase
    def exporter_cartes_phases(self, output_dir="cartes_phases", phases=(0, 1, 2, 3, 4)):
        """
        Crée une carte PNG par phase (0..4) :
          - infras contextuelles (toutes) en gris clair
          - infras réparées à la phase en couleur (selon type_infra)
          - bâtiments contextuels en gris
          - bâtiments de la phase en couleur (hôpital en étoile)
        """
        if self.gdf_infrastructures is None or self.gdf_batiments is None:
            print("Shapefiles non chargés : impossible de générer les cartes.")
            return

        os.makedirs(output_dir, exist_ok=True)

        #harmoniser CRS
        try:
            if self.gdf_infrastructures.crs and self.gdf_batiments.crs:
                if self.gdf_infrastructures.crs != self.gdf_batiments.crs:
                    self.gdf_batiments = self.gdf_batiments.to_crs(self.gdf_infrastructures.crs)
        except Exception as e:
            print(f"Attention CRS: {e}")

        # maps pour phases et types
        infra_phase_map = {i.id: (i.phase_reparation if i.phase_reparation is not None else -1)
                           for i in self.infrastructures.values()}
        bat_phase_map = {b.id: (b.phase if b.phase is not None else -1)
                         for b in self.batiments.values()}
        bat_type_map = {b.id: b.type_batiment for b in self.batiments.values()}
        infra_type_map = {i.id: i.type_infra for i in self.infrastructures.values()}

        gdf_inf = self.gdf_infrastructures.copy()
        gdf_bat = self.gdf_batiments.copy()

        infra_id_col = "infra_id"
        bat_id_col = "id_bat"

        gdf_inf["phase_reparation"] = gdf_inf[infra_id_col].astype(str).map(infra_phase_map).fillna(-1).astype(int)
        gdf_inf["type_infra"] = gdf_inf[infra_id_col].astype(str).map(infra_type_map).fillna("")

        gdf_bat["phase_bat"] = gdf_bat[bat_id_col].astype(str).map(bat_phase_map).fillna(-1).astype(int)
        gdf_bat["type_bat"] = gdf_bat[bat_id_col].astype(str).map(bat_type_map).fillna("")

        type_colors = {
            "aerien": "#1f77b4",       # bleu
            "semi-aerien": "#ff7f0e",  # orange
            "semi-aérien": "#ff7f0e",
            "fourreau": "#2ca02c",     # vert
            "": "#9467bd"              # fallback
        }

        for phase in phases:
            # contexte
            ax = gdf_inf.plot(figsize=(12, 10), color="#D9D9D9", linewidth=1.0, alpha=0.7)
            try:
                gdf_bat.plot(ax=ax, color="#BDBDBD", markersize=10, alpha=0.6)
            except Exception:
                pass

            # infras de la phase
            gdf_inf_phase = gdf_inf[gdf_inf["phase_reparation"] == phase]
            if not gdf_inf_phase.empty:
                for t, sub in gdf_inf_phase.groupby("type_infra"):
                    col = type_colors.get(t, "#9467bd")
                    sub.plot(ax=ax, color=col, linewidth=2.2, alpha=0.95, label=f"Infra {t} (phase {phase})")

            #batiments de la phase
            gdf_bat_phase = gdf_bat[gdf_bat["phase_bat"] == phase]
            if not gdf_bat_phase.empty:
                # vectorisation correcte
                types_norm = (
                    gdf_bat_phase["type_bat"]
                    .fillna("")
                    .astype(str)
                    .apply(_strip_accents_lower)
                )
                mask_hopital = types_norm == "hopital"

                gdf_bat_h = gdf_bat_phase[mask_hopital]
                gdf_bat_other = gdf_bat_phase[~mask_hopital]

                if not gdf_bat_other.empty:
                    gdf_bat_other.plot(ax=ax, color="#d62728", markersize=25, alpha=0.9,
                                       label=f"Bâtiments phase {phase}")
                if not gdf_bat_h.empty:
                    try:
                        gdf_bat_h.plot(ax=ax, color="#e377c2", markersize=80, marker="*",
                                       alpha=0.95, label="Hôpital")
                    except Exception:
                        gdf_bat_h.plot(ax=ax, color="#e377c2", markersize=50, alpha=0.95, label="Hôpital")

            ax.set_title(f"Plan de raccordement – Phase {phase}", fontsize=16, fontweight="bold")
            ax.set_axis_off()

            #legende 
            handles, labels = ax.get_legend_handles_labels()
            if handles:
                seen = {}
                new_handles, new_labels = [], []
                for h, l in zip(handles, labels):
                    if l not in seen:
                        seen[l] = True
                        new_handles.append(h)
                        new_labels.append(l)
                ax.legend(new_handles, new_labels, loc="lower left", frameon=True)

            out_path = os.path.join(output_dir, f"carte_phase_{phase}.png")
            fig = ax.get_figure()
            fig.savefig(out_path, dpi=300, bbox_inches="tight")
            fig.clf()
            print(f"Carte sauvegardée: {out_path}")


# Test
if __name__ == "__main__":
    xlsx_path = "reseau_en_arbre.xlsx"
    shp_infra_path = "infrastructures.shp"  
    shp_bat_path = "batiments.shp"         

    planif = PlanificateurRaccordement(xlsx_path, shp_infra_path, shp_bat_path,
                                       batiments_csv="batiments.csv", infra_csv="infra.csv")
    planif.charger_donnees()
    planif.planifier_raccordement()
    df_plan = planif.afficher_plan()
    df_plan.to_excel("plan_raccordement_phases.xlsx", index=False)
    print("\nTableau enregistré sous 'plan_raccordement_phases.xlsx'")

    # export des cartes par phase (0..4)
    planif.exporter_cartes_phases(output_dir="cartes_phases", phases=(0, 1, 2, 3, 4))


Chargement des données...
6107 lignes de réseau chargées
381 bâtiments chargés
644 infrastructures chargées

Début de la planification...

Planification terminée.
Coût total estimé : 1457263.57 €

Tableau du plan de raccordement :

 Phase Bâtiment Type bâtiment  Maisons  Difficulté                                       Infras réparées  Coût matériel (phase)  Coût MO (phase)  Coût total (phase)  Coût cumulé  Heures (phase)  Durée critique (h) Hopital_OK_≤16h
     0  E000085       hôpital        1   211214.33                             P005500, P007990, P007447               18483.26          2921.87            21405.13     21405.13           77.92                9.35              OK
     1  E000194    habitation        1    20670.55                    P005100, P000732, P008001, P000719               18773.43          2816.02            21589.45     42994.58           75.09                5.39             NaN
     1  E000195    habitation        1        0.00                            

Carte sauvegardée: cartes_phases/carte_phase_0.png
Carte sauvegardée: cartes_phases/carte_phase_1.png
Carte sauvegardée: cartes_phases/carte_phase_2.png
Carte sauvegardée: cartes_phases/carte_phase_3.png
Carte sauvegardée: cartes_phases/carte_phase_4.png


## Resume

In [10]:
def resume_par_phase(df: pd.DataFrame, budget_splits=(0.40,0.20,0.20,0.20)):
    out = []
    phases = sorted(df["Phase"].unique())
    # coût restant après phase 0 
    cout_total = df.loc[df["Phase"]>0, "Coût total (phase)"].sum() if "Coût total (phase)" in df.columns else None
    for ph in phases:
        d = df[df["Phase"]==ph].copy()
        bat_count = d["Bâtiment"].nunique()
        nb_maisons = d["Maisons"].sum()
        cmat = d.get("Coût matériel (phase)", pd.Series([0])).sum()
        cmo  = d.get("Coût MO (phase)", pd.Series([0])).sum()
        ctot = d.get("Coût total (phase)", pd.Series([0])).sum()
        heures = d.get("Heures (phase)", pd.Series([0])).sum()
        duree = d.get("Durée critique (h)", pd.Series([0])).max()

        budget_msg = ""
        if ph >= 1 and ph <= 4 and cout_total is not None:
            cap = budget_splits[ph-1]*cout_total
            budget_msg = f" | Budget cible phase {ph}: ~{cap:,.0f} € (tol. +3%)"
        if ph == 0 and "Hopital_OK_≤16h" in d.columns:
            ok = d["Hopital_OK_≤16h"].iloc[0]
            budget_msg = f" | Hôpital ≤16h: {ok}"

        out.append(
            f"Phase {ph} — {bat_count} bâtiments, {nb_maisons} ménages | "
            f"Matériel: {cmat:,.0f} € | MO: {cmo:,.0f} € | Total: {ctot:,.0f} € | "
            f"Heures: {heures:,.1f} h | Durée critique: {duree:,.1f} h{budget_msg}"
        )
    return "\n".join(out)

print(resume_par_phase(df_plan))


Phase 0 — 1 bâtiments, 1 ménages | Matériel: 18,483 € | MO: 2,922 € | Total: 21,405 € | Heures: 77.9 h | Durée critique: 9.3 h | Hôpital ≤16h: OK
Phase 1 — 47 bâtiments, 47 ménages | Matériel: 497,389 € | MO: 84,544 € | Total: 581,933 € | Heures: 2,254.5 h | Durée critique: 15.7 h | Budget cible phase 1: ~574,343 € (tol. +3%)
Phase 2 — 17 bâtiments, 17 ménages | Matériel: 248,821 € | MO: 46,690 € | Total: 295,511 € | Heures: 1,245.1 h | Durée critique: 38.2 h | Budget cible phase 2: ~287,172 € (tol. +3%)
Phase 3 — 11 bâtiments, 11 ménages | Matériel: 229,806 € | MO: 41,081 € | Total: 270,887 € | Heures: 1,095.5 h | Durée critique: 18.5 h | Budget cible phase 3: ~287,172 € (tol. +3%)
Phase 4 — 9 bâtiments, 9 ménages | Matériel: 240,836 € | MO: 46,691 € | Total: 287,528 € | Heures: 1,245.1 h | Durée critique: 43.3 h | Budget cible phase 4: ~287,172 € (tol. +3%)
